In [20]:
import os
import pandas as pd
from lxml import html
from bs4 import BeautifulSoup
import spacy
from boilerpipe.extract import Extractor
import codecs

nlp = spacy.load('en_core_web_sm')

In [39]:
os.chdir('C://Users//Ruben//Documents/GitHub/ReACT_GCV/code/work/scrape_environment/image_npg_1/html')

In [55]:
with open('C://Users//Ruben//Documents/GitHub/ReACT_GCV/code/work/scrape_environment/image_npg_1/html/results.txt') as f:
    resdf = f.readlines()
    resdf = [l.split('|') for l in resdf]
    resdf = [(r[0].split('html')[1][1:] + ".html",r[1].replace('\n','')) for r in resdf]

In [40]:
list_html = [i for i in os.listdir(os.getcwd()) if ".html" in i]

In [140]:
def ParserBoilerArticle(html_object):
    extractor = Extractor(extractor='ArticleSentencesExtractor', html=html_object)
    sents = extractor.getText()
    sents = list(nlp(sents).sents)
    return sents

def ParserBoilerDefault(html_object):
    extractor = Extractor(extractor='DefaultExtractor', html=html_object)
    sents = extractor.getText()
    sents = list(nlp(sents).sents)
    return sents

def ParserBoilerEverything(html_object):
    extractor = Extractor(extractor='DefaultExtractor', html=html_object)
    sents = extractor.getText()
    sents = list(nlp(sents).sents)
    return sents

def ParserRaw(html_object):
    soup = BeautifulSoup(html_object, "html.parser")
    [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
    text = soup.getText()
    text = [t for t in text if t]
    return " ".join(text)

In [141]:
line_len = pd.DataFrame()

for c,i in enumerate(list_html):
    with codecs.open(i,'r',encoding='utf-8',errors='ignore') as f:
        html_object = f.read()
    sents = ParserBoilerArticle(html_object)
    if len(sents) < 2:
        sents = ParserBoilerDefault(html_object)
    if len(sents) < 2:
        sents = ParserBoilerEverything(html_object)
    if len(sents) < 2:
        sents = ParserRaw(html_object)
        
        
        
    line_len = line_len.append(pd.DataFrame([i,len(sents),c,sents]).T)
    
line_len.columns = ['fn','length','ind','sents']

In [142]:
urls=[]
for c,i in enumerate(line_len['fn']):
    urls.append([x for x in resdf if x[0] == str(i)][0][1])
line_len['url'] = urls

In [143]:
len([i for i in line_len[line_len['length'] == 0]['url']])

6

In [150]:
for i in line_len[line_len['length'] < 4]['sents']:
    print(i)




[The girl in the photo: Kim Phuc has laser treatment for napalm burns – video
Kim Phuc arrives in Miami for dermatology treatment to reduce the pain she suffers from burns incurred in 1972 when she was nine years old., Phuc was photographed after being burnt in a napalm attack during the Vietnam war., Nick Ut, who took the photo, remains in touch and accompanies Phuc to her laser treatment, which will continue for several months
]
[Dziewczynka ze synnego zdjcia z Wietnamu po 40 latach pozbywa si blu., Usunie blizny po poparzeniach napalmem
Nick Ut / AP (AP Photo/Nick Ut, File)
Zdjcie 9-letniej Kim Phuc, uciekajcej ze swoim bratem przed samolotami wojsk Wietnamu Poudniowego., W Trang Bang, rodzinnej wiosce dziewczynki, dokonano pomykowego nalotu z uyciem poncego napalmu.
]


[挪威最有名的報紙Aftenposten的總編輯Espen Egil Hansen更出頭版公開信，公然指責他藉著Facebook成為了一個全球性的分享新聞資訊的平台，濫用權力，嚴重打擊民主社會的言論自由，令整個原應以民主為中心的社交平台變為一個獨裁的媒介, 。
, 標籤
]
[Telemetro es un canal de MEDCOM Panamá | Copyright © 2019., Todos los der

In [148]:
str(line_len[line_len['fn'] == "c90682b3-006b-4d50-8ad5-5e7f06f60e44.html"]['sents'])

'0    [(An, appropriate, representation, of, the, re...\nName: sents, dtype: object'